In [9]:
import numpy as np
#1. define categories and Questions
categories = ["Exoplanets", "Galaxies", "Constellations", "Planets", "Moons", "Astronomers/Physicists/Astronomers"]
# 5 rows = 
board_values = np.array([[100, 100, 100, 100, 100, 100], 
                         [200, 200, 200, 200, 200, 200], 
                         [300, 300, 300, 300, 300, 300], 
                         [400, 400, 400, 400, 400, 400], 
                         [500, 500, 500, 500, 500, 500] ])

# 2. Randomly pick a Daily Double location using choice
dd_selection = np.random.choice(30)

# Convert that one number into specific row (0-4) and col (0-5)
dd_row = dd_selection // 6
dd_col = dd_selection % 6

print(f"Daily Double Location: Row {dd_row}, Col {dd_col} (Category: {categories[dd_col]})")

# 3. Initialize Board with placeholders (5 rows x 6 columns)

ddArray = np.array(["Q" for _ in range(30)])
ddArray = ddArray.reshape(5, 6)
ddArray[dd_row][dd_col] = "DD" # Mark Daily Double

# 4. Display Board
print("\nJeopardy Board:")
print(ddArray)

list = []
for i in range(30):
    list.append(False)
beenUsed = np.array(list)
beenUsed = beenUsed.reshape(5, 6)
print(beenUsed)

Daily Double Location: Row 0, Col 4 (Category: Moons)

Jeopardy Board:
[['Q' 'Q' 'Q' 'Q' 'D' 'Q']
 ['Q' 'Q' 'Q' 'Q' 'Q' 'Q']
 ['Q' 'Q' 'Q' 'Q' 'Q' 'Q']
 ['Q' 'Q' 'Q' 'Q' 'Q' 'Q']
 ['Q' 'Q' 'Q' 'Q' 'Q' 'Q']]
[[False False False False False False]
 [False False False False False False]
 [False False False False False False]
 [False False False False False False]
 [False False False False False False]]


In [8]:
from clue_object_setup import Tile
from clue_object_setup import generate_tiles
from AI_Setup import create_clues
from AI_Setup import create_answers
from Play_Function import play
from AI_Setup import compare_answer
from player_object_setup import Player
from player_object_setup import startGame

answer = create_answers()
clues, answers = create_clues(answer)

board = generate_tiles(clues, answers, ddArray, board_values, beenUsed)



Clues generated successfully.
HD189733b


In [13]:


def assignClue(category, point_values):
    if category == "exoplanets":
        board_column = 0
    elif category == "galaxies":
        board_column = 1
    elif category == "constellations":
        board_column = 2
    elif category == "solar system":
        board_column = 3
    elif category == "moons":
        board_column = 4
    elif category == "astronomers":
        board_column = 5

    if point_values == "100":
        board_row = 0
    elif point_values == "200":
        board_row = 1
    elif point_values == "300":
        board_row = 2
    elif point_values == "400":
        board_row = 3
    elif point_values == "500":
        board_row = 4

    return board[board_row][board_column]

def getPlayerAnswer(chosenTileAnswer, chosenTileClue):
    userAnswer = input(f"{chosenTileClue}")
    output = compare_answer(chosenTileAnswer, userAnswer)
    return output

def editPoints(chosenTile, correctness, player):
    if correctness == "Correct":
        player.points += chosenTile.pointValue
    elif correctness == "Incorrect":
        player.points -= chosenTile.pointValue
    else:
        player.points = player.points
    
    return player.points



In [15]:


while True:

    player1 = startGame()
    
    

    while True:

        point_values, category = play()
        chosenTile = assignClue(category, point_values)
        if chosenTile.beenUsed:
            print("You have already selected this tile.")
            continue
        chosenTile.beenUsed = True

        print(chosenTile.clue)
        correctness = getPlayerAnswer(chosenTile.answer, chosenTile.clue)

        currentPoints = editPoints(chosenTile, correctness, player1)

        stop = input(f"You currently have {currentPoints} points. Press enter to continue, or type 'Y' to stop playing:")
        if stop == "Y":
            print(f"Thanks for playing!")
            break
        elif stop == "y":
            print(f"Thanks for playing!")
            break
        else:
            continue
    
    

    break



Hello, Welcome to AstroJeopardy! Please enter user information.

Host: Hey ryan! What kind of coding projects are you working on these days?

Host: That sounds awesome! What’s your favorite category or topic to include in your Jeopardy projects?

Host: Nice choice! Is there a particular planet or star that fascinates you the most?

Host: Great pick! Let's start playing!
You have already selected this tile.
This Hot Jupiter exoplanet, discovered via radial velocity, is infamous for its sideways rain of molten glass and an atmosphere rich in hydrogen sulfide, giving it a foul smell reminiscent of rotten eggs.
Please respond with your answer formatted as a question.


AttributeError: 'NoneType' object has no attribute 'points'

In [ ]:


while True:

    startGame()
    
    

    while True:

        point_values, category = play()
        chosenTile = assignClue(category, point_values)
        if chosenTile.beenUsed:
            print("You have already selected this tile.")
            continue
        chosenTile.beenUsed = True

        print(chosenTile.clue)
        correctness = getPlayerAnswer(chosenTile.answer, chosenTile.clue)



        stop = input("Type 'Y' to stop playing:")
        if stop == "Y":
            print(f"Thanks for playing!")
            break
        elif stop == "y":
            print(f"Thanks for playing!")
            break
        else:
            continue
    
    

    break



Correct
